# LangGraph బేసిక్స్

ఈ ట్యుటోరియల్‌లో, మనం LangChain లో LangGraph గురించి నేర్చుకుంటాము. LangGraph అనేది LangChain ఎకోసిస్టమ్‌లో ఒక భాగం, ఇది స్టేట్‌ఫుల్ మల్టి-ఏజెంట్ సిస్టమ్స్‌ని బిల్డ్ చేయడానికి ఉపయోగపడుతుంది.

## సెటప్

మొదట, మనం అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేసుకుందాం:

In [ ]:
# అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేయడం
!pip install langchain langchain-openai langgraph openai

ఇప్పుడు, మనం OpenAI API కీని సెట్ చేద్దాం:

In [ ]:
import os
from dotenv import load_dotenv

# .env ఫైల్ నుండి API కీని లోడ్ చేయడం
load_dotenv()

# లేదా డైరెక్ట్‌గా సెట్ చేయడం (డెవలప్‌మెంట్ కోసం మాత్రమే, ప్రొడక్షన్‌లో ఉపయోగించవద్దు)
# os.environ["OPENAI_API_KEY"] = "మీ-API-కీ-ఇక్కడ-పెట్టండి"

## LangGraph అంటే ఏమిటి?

LangGraph అనేది LangChain ఎకోసిస్టమ్‌లో ఒక భాగం, ఇది స్టేట్‌ఫుల్ మల్టి-ఏజెంట్ సిస్టమ్స్‌ని బిల్డ్ చేయడానికి ఉపయోగపడుతుంది. ఇది గ్రాఫ్ థియరీ కాన్సెప్ట్స్‌ని ఉపయోగించి, కాంప్లెక్స్ వర్క్‌ఫ్లోలను మాడల్ చేయడానికి మరియు ఎగ్జిక్యూట్ చేయడానికి సహాయపడుతుంది.

LangGraph ఎందుకు ముఖ్యమైనది?
1. **స్టేట్‌ఫుల్నెస్**: స్టేట్‌ని మెయింటెయిన్ చేయడం మరియు అప్‌డేట్ చేయడం
2. **కాంప్లెక్స్ వర్క్‌ఫ్లోలు**: బ్రాంచింగ్, లూపింగ్, మరియు కండిషనల్ లాజిక్‌తో కాంప్లెక్స్ వర్క్‌ఫ్లోలను మాడల్ చేయడం
3. **మల్టి-ఏజెంట్ సిస్టమ్స్**: వివిధ ఏజెంట్ల మధ్య కమ్యూనికేషన్ మరియు కోఆర్డినేషన్‌ని ఫెసిలిటేట్ చేయడం
4. **మాడ్యులారిటీ**: కాంప్లెక్స్ సిస్టమ్స్‌ని చిన్న, రీయూజబుల్ కంపోనెంట్స్‌గా విభజించడం

## LangGraph కీ కాన్సెప్ట్స్

LangGraph లో కొన్ని కీ కాన్సెప్ట్స్ ఉన్నాయి:

1. **నోడ్స్**: గ్రాఫ్‌లోని ప్రాసెసింగ్ యూనిట్స్, ఇవి ఫంక్షన్స్ లేదా LLM కాల్స్ అయి ఉండవచ్చు
2. **ఎడ్జెస్**: నోడ్స్ మధ్య కనెక్షన్స్, ఇవి డేటా ఫ్లో మరియు కంట్రోల్ ఫ్లోని డిఫైన్ చేస్తాయి
3. **స్టేట్**: గ్రాఫ్ ఎగ్జిక్యూషన్ సమయంలో మెయింటెయిన్ చేయబడే డేటా
4. **ట్రాన్సిషన్స్**: ఒక నోడ్ నుండి మరొక నోడ్‌కి ఎప్పుడు మూవ్ చేయాలో డిఫైన్ చేసే రూల్స్

## సింపుల్ LangGraph ఎగ్జాంపుల్

మొదట, మనం ఒక సింపుల్ LangGraph ఎగ్జాంపుల్‌ని చూద్దాం:

In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict, List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# స్టేట్ డిఫైన్ చేయడం
class MovieRecommendationState(TypedDict):
    genre: str
    language: str
    recommendations: List[Dict[str, Any]]

# నోడ్స్ డిఫైన్ చేయడం
def get_telugu_movies(state: MovieRecommendationState) -> MovieRecommendationState:
    """తెలుగు సినిమాలను పొందడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """తెలుగు {genre} సినిమాల జాబితాను అందించండి. ప్రతి సినిమా కోసం, టైటిల్, దర్శకుడు, ప్రధాన నటులు, విడుదల సంవత్సరం, మరియు చిన్న కథ సారాంశం అందించండి.
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        [
            {{
                "title": "సినిమా పేరు",
                "director": "దర్శకుడు పేరు",
                "actors": ["నటుడు 1", "నటుడు 2"],
                "year": విడుదల సంవత్సరం,
                "plot": "కథ సారాంశం"
            }},
            ...
        ]
        
        కనీసం 3 సినిమాలను అందించండి.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({"genre": state["genre"]})
    
    # JSON పార్స్ చేయడం
    import json
    recommendations = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "recommendations": recommendations}

def get_hindi_movies(state: MovieRecommendationState) -> MovieRecommendationState:
    """హిందీ సినిమాలను పొందడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """హిందీ {genre} సినిమాల జాబితాను అందించండి. ప్రతి సినిమా కోసం, టైటిల్, దర్శకుడు, ప్రధాన నటులు, విడుదల సంవత్సరం, మరియు చిన్న కథ సారాంశం అందించండి.
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        [
            {{
                "title": "సినిమా పేరు",
                "director": "దర్శకుడు పేరు",
                "actors": ["నటుడు 1", "నటుడు 2"],
                "year": విడుదల సంవత్సరం,
                "plot": "కథ సారాంశం"
            }},
            ...
        ]
        
        కనీసం 3 సినిమాలను అందించండి.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({"genre": state["genre"]})
    
    # JSON పార్స్ చేయడం
    import json
    recommendations = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "recommendations": recommendations}

def decide_language(state: MovieRecommendationState) -> str:
    """భాష ఆధారంగా నెక్స్ట్ నోడ్‌ని నిర్ణయించడం"""
    if state["language"].lower() == "telugu":
        return "telugu"
    else:
        return "hindi"

# గ్రాఫ్ క్రియేట్ చేయడం
workflow = StateGraph(MovieRecommendationState)

# నోడ్స్ యాడ్ చేయడం
workflow.add_node("telugu", get_telugu_movies)
workflow.add_node("hindi", get_hindi_movies)

# ఎడ్జెస్ యాడ్ చేయడం
workflow.add_conditional_edges(
    "start",
    decide_language,
    {
        "telugu": "telugu",
        "hindi": "hindi"
    }
)

# ఎండ్ నోడ్స్ సెట్ చేయడం
workflow.add_edge("telugu", "end")
workflow.add_edge("hindi", "end")

# గ్రాఫ్‌ని కంపైల్ చేయడం
app = workflow.compile()

# గ్రాఫ్‌ని రన్ చేయడం
for output in app.stream({"genre": "యాక్షన్", "language": "telugu", "recommendations": []}):
    if "recommendations" in output:
        print("\nసినిమా సిఫార్సులు:")
        for i, movie in enumerate(output["recommendations"]):
            print(f"\n{i+1}. {movie['title']} ({movie['year']})")
            print(f"   దర్శకుడు: {movie['director']}")
            print(f"   నటులు: {', '.join(movie['actors'])}")
            print(f"   కథ: {movie['plot'][:100]}...")

## మల్టి-స్టెప్ LangGraph ఎగ్జాంపుల్

ఇప్పుడు, మనం ఒక మల్టి-స్టెప్ LangGraph ఎగ్జాంపుల్‌ని చూద్దాం:

In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict, List, Dict, Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# స్టేట్ డిఫైన్ చేయడం
class TravelPlanState(TypedDict):
    destination: str
    days: int
    budget: int
    interests: List[str]
    attractions: List[Dict[str, Any]]
    hotels: List[Dict[str, Any]]
    restaurants: List[Dict[str, Any]]
    itinerary: Optional[List[Dict[str, Any]]]
    final_plan: Optional[str]

# నోడ్స్ డిఫైన్ చేయడం
def get_attractions(state: TravelPlanState) -> TravelPlanState:
    """ఆకర్షణీయ ప్రదేశాలను పొందడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """ప్రయాణికుడు {destination}కి వెళ్తున్నారు. వారి ఆసక్తులు: {interests}. {destination}లో టాప్ ఆకర్షణీయ ప్రదేశాల జాబితాను అందించండి.
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        [
            {{
                "name": "ఆకర్షణీయ ప్రదేశం పేరు",
                "description": "వివరణ",
                "category": "కేటగిరీ (ఉదా. చారిత్రక, ప్రకృతి, సాంస్కృతిక, మొదలైనవి)",
                "time_required": "సందర్శించడానికి అవసరమైన సమయం (గంటల్లో)",
                "cost": సందర్శన ఖర్చు (రూపాయలలో, అంచనా)
            }},
            ...
        ]
        
        కనీసం 5 ఆకర్షణీయ ప్రదేశాలను అందించండి.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({"destination": state["destination"], "interests": ", ".join(state["interests"])})
    
    # JSON పార్స్ చేయడం
    import json
    attractions = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "attractions": attractions}

def get_hotels(state: TravelPlanState) -> TravelPlanState:
    """హోటల్స్‌ని పొందడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """ప్రయాణికుడు {destination}కి వెళ్తున్నారు. వారి బడ్జెట్: {budget} రూపాయలు. {destination}లో టాప్ హోటల్స్ జాబితాను అందించండి.
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        [
            {{
                "name": "హోటల్ పేరు",
                "description": "వివరణ",
                "category": "కేటగిరీ (ఉదా. లగ్జరీ, మిడ్-రేంజ్, బడ్జెట్, మొదలైనవి)",
                "cost_per_night": ఒక రాత్రికి ఖర్చు (రూపాయలలో, అంచనా),
                "amenities": ["అమెనిటీ 1", "అమెనిటీ 2", ...],
                "location": "లొకేషన్ వివరాలు"
            }},
            ...
        ]
        
        కనీసం 3 హోటల్స్‌ని అందించండి, వివిధ బడ్జెట్ రేంజ్‌లలో.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({"destination": state["destination"], "budget": state["budget"]})
    
    # JSON పార్స్ చేయడం
    import json
    hotels = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "hotels": hotels}

def get_restaurants(state: TravelPlanState) -> TravelPlanState:
    """రెస్టారెంట్స్‌ని పొందడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """ప్రయాణికుడు {destination}కి వెళ్తున్నారు. {destination}లో టాప్ రెస్టారెంట్స్ జాబితాను అందించండి.
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        [
            {{
                "name": "రెస్టారెంట్ పేరు",
                "description": "వివరణ",
                "cuisine": "వంటకాల రకం",
                "cost_for_two": ఇద్దరికి ఖర్చు (రూపాయలలో, అంచనా),
                "must_try_dishes": ["డిష్ 1", "డిష్ 2", ...],
                "location": "లొకేషన్ వివరాలు"
            }},
            ...
        ]
        
        కనీసం 5 రెస్టారెంట్స్‌ని అందించండి, వివిధ వంటకాల రకాలతో.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({"destination": state["destination"]})
    
    # JSON పార్స్ చేయడం
    import json
    restaurants = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "restaurants": restaurants}

def create_itinerary(state: TravelPlanState) -> TravelPlanState:
    """ఇటినరరీని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """ప్రయాణికుడు {destination}కి {days} రోజుల పాటు వెళ్తున్నారు. వారి ఆసక్తులు: {interests}. వారి బడ్జెట్: {budget} రూపాయలు.
        
        ఆకర్షణీయ ప్రదేశాలు:
        {attractions}
        
        హోటల్స్:
        {hotels}
        
        రెస్టారెంట్స్:
        {restaurants}
        
        పై సమాచారం ఆధారంగా, {days} రోజుల ఇటినరరీని క్రియేట్ చేయండి.
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        [
            {{
                "day": 1,
                "morning": {{
                    "activity": "యాక్టివిటీ వివరణ",
                    "location": "లొకేషన్",
                    "time": "సమయం"
                }},
                "afternoon": {{
                    "activity": "యాక్టివిటీ వివరణ",
                    "location": "లొకేషన్",
                    "time": "సమయం"
                }},
                "evening": {{
                    "activity": "యాక్టివిటీ వివరణ",
                    "location": "లొకేషన్",
                    "time": "సమయం"
                }},
                "dinner": {{
                    "restaurant": "రెస్టారెంట్ పేరు",
                    "time": "సమయం"
                }},
                "accommodation": "హోటల్ పేరు"
            }},
            ...
        ]
        
        ప్రతి రోజుకి ఒక ఆబ్జెక్ట్ క్రియేట్ చేయండి, మొత్తం {days} రోజులకు.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "destination": state["destination"],
        "days": state["days"],
        "budget": state["budget"],
        "interests": ", ".join(state["interests"]),
        "attractions": json.dumps(state["attractions"], ensure_ascii=False),
        "hotels": json.dumps(state["hotels"], ensure_ascii=False),
        "restaurants": json.dumps(state["restaurants"], ensure_ascii=False)
    })
    
    # JSON పార్స్ చేయడం
    import json
    itinerary = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "itinerary": itinerary}

def create_final_plan(state: TravelPlanState) -> TravelPlanState:
    """ఫైనల్ ప్లాన్‌ని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """ప్రయాణికుడు {destination}కి {days} రోజుల పాటు వెళ్తున్నారు. వారి ఆసక్తులు: {interests}. వారి బడ్జెట్: {budget} రూపాయలు.
        
        ఇటినరరీ:
        {itinerary}
        
        పై సమాచారం ఆధారంగా, ప్రయాణికుడికి ఒక సంపూర్ణ ప్రయాణ ప్లాన్‌ని క్రియేట్ చేయండి. ఈ ప్లాన్‌లో ప్రతి రోజు యాక్టివిటీలు, భోజనం, అకమడేషన్, ట్రాన్స్‌పోర్టేషన్, మరియు ఇతర ముఖ్యమైన సమాచారం ఉండాలి.
        
        ప్లాన్‌ని తెలుగులో రాయండి, ప్రయాణికుడికి అర్థమయ్యే విధంగా.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "destination": state["destination"],
        "days": state["days"],
        "budget": state["budget"],
        "interests": ", ".join(state["interests"]),
        "itinerary": json.dumps(state["itinerary"], ensure_ascii=False)
    })
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "final_plan": response.content}

# గ్రాఫ్ క్రియేట్ చేయడం
workflow = StateGraph(TravelPlanState)

# నోడ్స్ యాడ్ చేయడం
workflow.add_node("get_attractions", get_attractions)
workflow.add_node("get_hotels", get_hotels)
workflow.add_node("get_restaurants", get_restaurants)
workflow.add_node("create_itinerary", create_itinerary)
workflow.add_node("create_final_plan", create_final_plan)

# ఎడ్జెస్ యాడ్ చేయడం
workflow.add_edge("start", "get_attractions")
workflow.add_edge("get_attractions", "get_hotels")
workflow.add_edge("get_hotels", "get_restaurants")
workflow.add_edge("get_restaurants", "create_itinerary")
workflow.add_edge("create_itinerary", "create_final_plan")
workflow.add_edge("create_final_plan", "end")

# గ్రాఫ్‌ని కంపైల్ చేయడం
app = workflow.compile()

# గ్రాఫ్‌ని రన్ చేయడం
inputs = {
    "destination": "హైదరాబాద్",
    "days": 3,
    "budget": 20000,
    "interests": ["చారిత్రక ప్రదేశాలు", "స్థానిక వంటకాలు", "షాపింగ్"],
    "attractions": [],
    "hotels": [],
    "restaurants": [],
    "itinerary": None,
    "final_plan": None
}

for output in app.stream(inputs):
    if "final_plan" in output and output["final_plan"] is not None:
        print("\nఫైనల్ ప్రయాణ ప్లాన్:")
        print(output["final_plan"])

## మల్టి-ఏజెంట్ LangGraph ఎగ్జాంపుల్

ఇప్పుడు, మనం ఒక మల్టి-ఏజెంట్ LangGraph ఎగ్జాంపుల్‌ని చూద్దాం:

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict, Any, Literal, Union, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# స్టేట్ డిఫైన్ చేయడం
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage, SystemMessage]]
    next: str

# ఏజెంట్స్ డిఫైన్ చేయడం
def create_agent(name: str, description: str, tools: List[str] = None):
    """ఏజెంట్‌ని క్రియేట్ చేయడం"""
    tools_str = "\n".join(tools) if tools else ""
    system_message = f"""మీరు {name}. {description}
    
    {tools_str}
    
    మీరు ఇతర ఏజెంట్లతో సహకరించి, యూజర్ ప్రశ్నకు సమాధానం ఇవ్వాలి.
    """
    
    def agent_function(state: AgentState):
        llm = ChatOpenAI(model="gpt-3.5-turbo")
        messages = state["messages"]
        response = llm.invoke([SystemMessage(content=system_message)] + messages)
        return {"messages": messages + [response]}
    
    return agent_function

# ఏజెంట్స్ క్రియేట్ చేయడం
movie_critic = create_agent(
    name="తెలుగు సినిమా విమర్శకుడు",
    description="మీరు తెలుగు సినిమాల గురించి లోతైన జ్ఞానం కలిగిన విమర్శకుడు. మీరు తెలుగు సినిమా చరిత్ర, ప్రముఖ దర్శకులు, నటులు, మరియు సినిమా ట్రెండ్స్ గురించి నిపుణుడు."
)

cricket_expert = create_agent(
    name="క్రికెట్ నిపుణుడు",
    description="మీరు క్రికెట్ గురించి లోతైన జ్ఞానం కలిగిన నిపుణుడు. మీరు IPL, టెస్ట్ క్రికెట్, మరియు క్రికెట్ ఆటగాళ్ల గురించి నిపుణుడు."
)

politics_analyst = create_agent(
    name="రాజకీయ విశ్లేషకుడు",
    description="మీరు తెలుగు రాష్ట్రాల రాజకీయాల గురించి లోతైన జ్ఞానం కలిగిన విశ్లేషకుడు. మీరు తెలుగు రాష్ట్రాల రాజకీయ చరిత్ర, ప్రముఖ నాయకులు, పార్టీలు, మరియు ప్రస్తుత రాజకీయ పరిస్థితుల గురించి నిపుణుడు."
)

coordinator = create_agent(
    name="సమన్వయకర్త",
    description="మీరు ఇతర ఏజెంట్ల మధ్య సమన్వయం చేసే సమన్వయకర్త. మీరు ప్రశ్నను విశ్లేషించి, సరైన ఏజెంట్‌ని ఎంచుకుని, అంతిమ సమాధానాన్ని సమన్వయం చేయాలి."
)

# రౌటర్ ఫంక్షన్ డిఫైన్ చేయడం
def router(state: AgentState) -> Literal["movie_critic", "cricket_expert", "politics_analyst", "coordinator", END]:
    """నెక్స్ట్ ఏజెంట్‌ని నిర్ణయించడం"""
    if state["next"] == "movie_critic":
        return "movie_critic"
    elif state["next"] == "cricket_expert":
        return "cricket_expert"
    elif state["next"] == "politics_analyst":
        return "politics_analyst"
    elif state["next"] == "coordinator":
        return "coordinator"
    else:
        return END

# ప్రశ్నను విశ్లేషించి, సరైన ఏజెంట్‌ని నిర్ణయించడం
def analyze_question(state: AgentState) -> AgentState:
    """ప్రశ్నను విశ్లేషించి, సరైన ఏజెంట్‌ని నిర్ణయించడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """ఈ ప్రశ్నను విశ్లేషించి, ఈ క్రింది ఏజెంట్లలో ఏ ఏజెంట్ సమాధానం ఇవ్వగలదో నిర్ణయించండి:
        
        1. movie_critic: తెలుగు సినిమాలకు సంబంధించిన ప్రశ్నలు
        2. cricket_expert: క్రికెట్‌కు సంబంధించిన ప్రశ్నలు
        3. politics_analyst: తెలుగు రాష్ట్రాల రాజకీయాలకు సంబంధించిన ప్రశ్నలు
        4. coordinator: ఏ ఏజెంట్‌కీ స్పష్టంగా సంబంధం లేని ప్రశ్నలు లేదా బహుళ ఏజెంట్ల నుండి సమాధానాలు అవసరమైన ప్రశ్నలు
        
        ప్రశ్న: {question}
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: మీ సమాధానం ఈ క్రింది ఫార్మాట్‌లో ఉండాలి:
        ```
        {{"agent": "selected_agent"}}
        ```
        """
    )
    
    chain = prompt | llm
    question = state["messages"][-1].content
    response = chain.invoke({"question": question})
    
    # JSON పార్స్ చేయడం
    import json
    result = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "next": result["agent"]}

# ఫైనల్ సమాధానాన్ని క్రియేట్ చేయడం
def create_final_answer(state: AgentState) -> AgentState:
    """ఫైనల్ సమాధానాన్ని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు ఒక సమన్వయకర్త. ఈ క్రింది సంభాషణను విశ్లేషించి, యూజర్ ప్రశ్నకు ఒక సంపూర్ణ సమాధానాన్ని క్రియేట్ చేయండి.
        
        సంభాషణ:
        {conversation}
        
        ఫైనల్ సమాధానాన్ని క్రియేట్ చేయండి. సమాధానం సంపూర్ణంగా, సమగ్రంగా, మరియు యూజర్‌కి అర్థమయ్యే విధంగా ఉండాలి.
        """
    )
    
    chain = prompt | llm
    conversation = "\n".join([f"{msg.__class__.__name__}: {msg.content}" for msg in state["messages"]])
    response = chain.invoke({"conversation": conversation})
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {"messages": state["messages"] + [response], "next": "end"}

# గ్రాఫ్ క్రియేట్ చేయడం
workflow = StateGraph(AgentState)

# నోడ్స్ యాడ్ చేయడం
workflow.add_node("analyze_question", analyze_question)
workflow.add_node("movie_critic", movie_critic)
workflow.add_node("cricket_expert", cricket_expert)
workflow.add_node("politics_analyst", politics_analyst)
workflow.add_node("coordinator", coordinator)
workflow.add_node("create_final_answer", create_final_answer)

# ఎడ్జెస్ యాడ్ చేయడం
workflow.add_edge("start", "analyze_question")
workflow.add_conditional_edges("analyze_question", router, {
    "movie_critic": "movie_critic",
    "cricket_expert": "cricket_expert",
    "politics_analyst": "politics_analyst",
    "coordinator": "coordinator"
})
workflow.add_edge("movie_critic", "create_final_answer")
workflow.add_edge("cricket_expert", "create_final_answer")
workflow.add_edge("politics_analyst", "create_final_answer")
workflow.add_edge("coordinator", "create_final_answer")
workflow.add_edge("create_final_answer", "end")

# గ్రాఫ్‌ని కంపైల్ చేయడం
app = workflow.compile()

# గ్రాఫ్‌ని రన్ చేయడం
def ask_question(question: str):
    """ప్రశ్న అడగడం"""
    inputs = {
        "messages": [HumanMessage(content=question)],
        "next": ""
    }
    
    for output in app.stream(inputs):
        if len(output["messages"]) > 1 and isinstance(output["messages"][-1], AIMessage) and output["next"] == "end":
            return output["messages"][-1].content

# వేర్వేరు ప్రశ్నలు అడగడం
questions = [
    "RRR సినిమా గురించి చెప్పండి",
    "IPL లో సన్‌రైజర్స్ హైదరాబాద్ జట్టు గురించి చెప్పండి",
    "తెలుగు రాష్ట్రాల ప్రస్తుత ముఖ్యమంత్రులు ఎవరు?",
    "తెలుగు సినిమాలు మరియు రాజకీయాల మధ్య సంబంధం ఏమిటి?"
]

for i, question in enumerate(questions):
    print(f"\nప్రశ్న {i+1}: {question}")
    answer = ask_question(question)
    print(f"సమాధానం: {answer}")

## రియల్-వరల్డ్ ఉదాహరణ: తెలుగు సినిమా స్క్రిప్ట్ జనరేటర్

ఇప్పుడు, మనం LangGraph ని ఉపయోగించి ఒక రియల్-వరల్డ్ ఉదాహరణను చూద్దాం: తెలుగు సినిమా స్క్రిప్ట్ జనరేటర్.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict, Any, Literal, Union, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import json

# స్టేట్ డిఫైన్ చేయడం
class ScriptGeneratorState(TypedDict):
    genre: str
    theme: str
    setting: str
    characters: Optional[List[Dict[str, Any]]]
    plot_outline: Optional[str]
    act1: Optional[str]
    act2: Optional[str]
    act3: Optional[str]
    dialogues: Optional[Dict[str, List[str]]]
    final_script: Optional[str]
    feedback: Optional[str]
    revision_needed: bool

# నోడ్స్ డిఫైన్ చేయడం
def create_characters(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """కథాపాత్రలను క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా స్క్రిప్ట్ రైటర్. ఈ క్రింది ఇన్‌పుట్స్ ఆధారంగా, ఒక తెలుగు సినిమా కోసం కథాపాత్రలను క్రియేట్ చేయండి:
        
        జానర్: {genre}
        థీమ్: {theme}
        సెట్టింగ్: {setting}
        
        కనీసం 5 ప్రధాన కథాపాత్రలను క్రియేట్ చేయండి, ప్రతి కథాపాత్ర కోసం పేరు, వయసు, వృత్తి, వ్యక్తిత్వం, లక్ష్యాలు, మరియు కథలో పాత్ర అందించండి.
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        [
            {{
                "name": "కథాపాత్ర పేరు",
                "age": వయసు (నంబర్),
                "occupation": "వృత్తి",
                "personality": "వ్యక్తిత్వం",
                "goals": "లక్ష్యాలు",
                "role": "కథలో పాత్ర (ప్రధాన నాయకుడు, నాయిక, విలన్, మొదలైనవి)",
                "background": "కథాపాత్ర నేపథ్యం"
            }},
            ...
        ]
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "genre": state["genre"],
        "theme": state["theme"],
        "setting": state["setting"]
    })
    
    # JSON పార్స్ చేయడం
    characters = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "characters": characters}

def create_plot_outline(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """ప్లాట్ అవుట్‌లైన్‌ని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా స్క్రిప్ట్ రైటర్. ఈ క్రింది ఇన్‌పుట్స్ ఆధారంగా, ఒక తెలుగు సినిమా కోసం ప్లాట్ అవుట్‌లైన్‌ని క్రియేట్ చేయండి:
        
        జానర్: {genre}
        థీమ్: {theme}
        సెట్టింగ్: {setting}
        కథాపాత్రలు: {characters}
        
        ప్లాట్ అవుట్‌లైన్‌లో ఇంట్రడక్షన్, కాన్ఫ్లిక్ట్, క్లైమాక్స్, మరియు రెజల్యూషన్ ఉండాలి. 3-యాక్ట్ స్ట్రక్చర్‌ని ఫాలో అవ్వండి.
        
        ప్లాట్ అవుట్‌లైన్‌ని తెలుగులో రాయండి, కనీసం 500 పదాలు ఉండాలి.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "genre": state["genre"],
        "theme": state["theme"],
        "setting": state["setting"],
        "characters": json.dumps(state["characters"], ensure_ascii=False)
    })
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "plot_outline": response.content}

def create_act1(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """యాక్ట్ 1 ని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా స్క్రిప్ట్ రైటర్. ఈ క్రింది ప్లాట్ అవుట్‌లైన్ ఆధారంగా, యాక్ట్ 1 ని క్రియేట్ చేయండి:
        
        ప్లాట్ అవుట్‌లైన్: {plot_outline}
        
        యాక్ట్ 1 లో కథాపాత్రల పరిచయం, సెట్టింగ్ ఎస్టాబ్లిష్‌మెంట్, మరియు ప్రధాన కాన్ఫ్లిక్ట్ ఇంట్రడక్షన్ ఉండాలి.
        
        స్క్రిప్ట్‌ని తెలుగులో రాయండి, స్క్రిప్ట్ ఫార్మాట్‌లో (సీన్ హెడింగ్స్, యాక్షన్ డిస్క్రిప్షన్స్, మరియు డైలాగ్స్‌తో).
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "plot_outline": state["plot_outline"]
    })
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "act1": response.content}

def create_act2(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """యాక్ట్ 2 ని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా స్క్రిప్ట్ రైటర్. ఈ క్రింది ప్లాట్ అవుట్‌లైన్ మరియు యాక్ట్ 1 ఆధారంగా, యాక్ట్ 2 ని క్రియేట్ చేయండి:
        
        ప్లాట్ అవుట్‌లైన్: {plot_outline}
        యాక్ట్ 1: {act1}
        
        యాక్ట్ 2 లో కాన్ఫ్లిక్ట్ డెవలప్‌మెంట్, అబ్స్టాకిల్స్, మరియు మిడ్‌పాయింట్ ఉండాలి.
        
        స్క్రిప్ట్‌ని తెలుగులో రాయండి, స్క్రిప్ట్ ఫార్మాట్‌లో (సీన్ హెడింగ్స్, యాక్షన్ డిస్క్రిప్షన్స్, మరియు డైలాగ్స్‌తో).
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "plot_outline": state["plot_outline"],
        "act1": state["act1"]
    })
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "act2": response.content}

def create_act3(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """యాక్ట్ 3 ని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా స్క్రిప్ట్ రైటర్. ఈ క్రింది ప్లాట్ అవుట్‌లైన్, యాక్ట్ 1, మరియు యాక్ట్ 2 ఆధారంగా, యాక్ట్ 3 ని క్రియేట్ చేయండి:
        
        ప్లాట్ అవుట్‌లైన్: {plot_outline}
        యాక్ట్ 1: {act1_summary}
        యాక్ట్ 2: {act2_summary}
        
        యాక్ట్ 3 లో క్లైమాక్స్, రెజల్యూషన్, మరియు ఎండింగ్ ఉండాలి.
        
        స్క్రిప్ట్‌ని తెలుగులో రాయండి, స్క్రిప్ట్ ఫార్మాట్‌లో (సీన్ హెడింగ్స్, యాక్షన్ డిస్క్రిప్షన్స్, మరియు డైలాగ్స్‌తో).
        """
    )
    
    # యాక్ట్ 1 మరియు 2 సమ్మరీలను క్రియేట్ చేయడం
    summarize_prompt = ChatPromptTemplate.from_template("ఈ స్క్రిప్ట్‌ని 100 పదాలలో సమ్మరైజ్ చేయండి:\n\n{script}")
    summarize_chain = summarize_prompt | llm
    
    act1_summary = summarize_chain.invoke({"script": state["act1"]}).content
    act2_summary = summarize_chain.invoke({"script": state["act2"]}).content
    
    chain = prompt | llm
    response = chain.invoke({
        "plot_outline": state["plot_outline"],
        "act1_summary": act1_summary,
        "act2_summary": act2_summary
    })
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "act3": response.content}

def create_dialogues(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """డైలాగ్స్‌ని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా డైలాగ్ రైటర్. ఈ క్రింది కథాపాత్రల ఆధారంగా, ప్రతి కథాపాత్ర కోసం 5 మెమరబుల్ డైలాగ్స్‌ని క్రియేట్ చేయండి:
        
        కథాపాత్రలు: {characters}
        ప్లాట్ అవుట్‌లైన్: {plot_outline}
        
        డైలాగ్స్‌ని తెలుగులో రాయండి, ప్రతి కథాపాత్ర వ్యక్తిత్వానికి తగినట్లుగా.
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        {{
            "character_name_1": ["డైలాగ్ 1", "డైలాగ్ 2", ...],
            "character_name_2": ["డైలాగ్ 1", "డైలాగ్ 2", ...],
            ...
        }}
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "characters": json.dumps(state["characters"], ensure_ascii=False),
        "plot_outline": state["plot_outline"]
    })
    
    # JSON పార్స్ చేయడం
    dialogues = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "dialogues": dialogues}

def create_final_script(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """ఫైనల్ స్క్రిప్ట్‌ని క్రియేట్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా స్క్రిప్ట్ రైటర్. ఈ క్రింది ఇన్‌పుట్స్ ఆధారంగా, ఒక సంపూర్ణ తెలుగు సినిమా స్క్రిప్ట్‌ని క్రియేట్ చేయండి:
        
        జానర్: {genre}
        థీమ్: {theme}
        సెట్టింగ్: {setting}
        కథాపాత్రలు: {characters}
        ప్లాట్ అవుట్‌లైన్: {plot_outline}
        యాక్ట్ 1: {act1_summary}
        యాక్ట్ 2: {act2_summary}
        యాక్ట్ 3: {act3_summary}
        డైలాగ్స్: {dialogues}
        
        ఫైనల్ స్క్రిప్ట్‌ని తెలుగులో రాయండి, ప్రొఫెషనల్ స్క్రిప్ట్ ఫార్మాట్‌లో. స్క్రిప్ట్‌లో టైటిల్ పేజీ, కథాపాత్రల జాబితా, మరియు సీన్‌లు ఉండాలి. ప్రతి సీన్‌లో సీన్ హెడింగ్, యాక్షన్ డిస్క్రిప్షన్స్, మరియు డైలాగ్స్ ఉండాలి.
        
        ఇప్పటికే క్రియేట్ చేసిన డైలాగ్స్‌ని ఉపయోగించండి, కానీ అవసరమైతే కొత్త డైలాగ్స్‌ని కూడా యాడ్ చేయండి.
        """
    )
    
    # యాక్ట్స్ సమ్మరీలను క్రియేట్ చేయడం
    summarize_prompt = ChatPromptTemplate.from_template("ఈ స్క్రిప్ట్‌ని 100 పదాలలో సమ్మరైజ్ చేయండి:\n\n{script}")
    summarize_chain = summarize_prompt | llm
    
    act1_summary = summarize_chain.invoke({"script": state["act1"]}).content
    act2_summary = summarize_chain.invoke({"script": state["act2"]}).content
    act3_summary = summarize_chain.invoke({"script": state["act3"]}).content
    
    chain = prompt | llm
    response = chain.invoke({
        "genre": state["genre"],
        "theme": state["theme"],
        "setting": state["setting"],
        "characters": json.dumps(state["characters"], ensure_ascii=False),
        "plot_outline": state["plot_outline"],
        "act1_summary": act1_summary,
        "act2_summary": act2_summary,
        "act3_summary": act3_summary,
        "dialogues": json.dumps(state["dialogues"], ensure_ascii=False)
    })
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "final_script": response.content}

def review_script(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """స్క్రిప్ట్‌ని రివ్యూ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా స్క్రిప్ట్ ఎడిటర్. ఈ క్రింది స్క్రిప్ట్‌ని రివ్యూ చేయండి:
        
        జానర్: {genre}
        థీమ్: {theme}
        సెట్టింగ్: {setting}
        
        స్క్రిప్ట్: {script}
        
        స్క్రిప్ట్‌ని ఈ క్రింది అంశాల ఆధారంగా రివ్యూ చేయండి:
        1. స్టోరీ స్ట్రక్చర్
        2. కథాపాత్రల డెవలప్‌మెంట్
        3. డైలాగ్స్
        4. పేస్
        5. థీమ్ మరియు జానర్‌కి అనుగుణంగా ఉందా
        
        ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫార్మాట్‌లో:
        {{
            "story_structure": "స్టోరీ స్ట్రక్చర్ రివ్యూ",
            "character_development": "కథాపాత్రల డెవలప్‌మెంట్ రివ్యూ",
            "dialogues": "డైలాగ్స్ రివ్యూ",
            "pacing": "పేస్ రివ్యూ",
            "theme_genre_alignment": "థీమ్ మరియు జానర్ అలైన్‌మెంట్ రివ్యూ",
            "overall_rating": 1-10 స్కేల్‌లో ఓవరాల్ రేటింగ్,
            "revision_needed": true/false,
            "revision_suggestions": "రివిజన్ సజెషన్స్"
        }}
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "genre": state["genre"],
        "theme": state["theme"],
        "setting": state["setting"],
        "script": state["final_script"]
    })
    
    # JSON పార్స్ చేయడం
    feedback = json.loads(response.content)
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "feedback": feedback, "revision_needed": feedback["revision_needed"]}

def revise_script(state: ScriptGeneratorState) -> ScriptGeneratorState:
    """స్క్రిప్ట్‌ని రివైజ్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు తెలుగు సినిమా స్క్రిప్ట్ రైటర్. ఈ క్రింది స్క్రిప్ట్‌ని ఫీడ్‌బ్యాక్ ఆధారంగా రివైజ్ చేయండి:
        
        స్క్రిప్ట్: {script}
        
        ఫీడ్‌బ్యాక్: {feedback}
        
        రివిజన్ సజెషన్స్‌ని ఫాలో చేస్తూ, స్క్రిప్ట్‌ని రివైజ్ చేయండి. రివైజ్డ్ స్క్రిప్ట్‌ని తెలుగులో రాయండి, ప్రొఫెషనల్ స్క్రిప్ట్ ఫార్మాట్‌లో.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "script": state["final_script"],
        "feedback": json.dumps(state["feedback"], ensure_ascii=False)
    })
    
    # స్టేట్‌ని అప్‌డేట్ చేయడం
    return {**state, "final_script": response.content, "revision_needed": False}

# రౌటర్ ఫంక్షన్ డిఫైన్ చేయడం
def revision_router(state: ScriptGeneratorState) -> Literal["revise_script", END]:
    """రివిజన్ అవసరమా కాదా అని నిర్ణయించడం"""
    if state["revision_needed"]:
        return "revise_script"
    else:
        return END

# గ్రాఫ్ క్రియేట్ చేయడం
workflow = StateGraph(ScriptGeneratorState)

# నోడ్స్ యాడ్ చేయడం
workflow.add_node("create_characters", create_characters)
workflow.add_node("create_plot_outline", create_plot_outline)
workflow.add_node("create_act1", create_act1)
workflow.add_node("create_act2", create_act2)
workflow.add_node("create_act3", create_act3)
workflow.add_node("create_dialogues", create_dialogues)
workflow.add_node("create_final_script", create_final_script)
workflow.add_node("review_script", review_script)
workflow.add_node("revise_script", revise_script)

# ఎడ్జెస్ యాడ్ చేయడం
workflow.add_edge("start", "create_characters")
workflow.add_edge("create_characters", "create_plot_outline")
workflow.add_edge("create_plot_outline", "create_act1")
workflow.add_edge("create_act1", "create_act2")
workflow.add_edge("create_act2", "create_act3")
workflow.add_edge("create_act3", "create_dialogues")
workflow.add_edge("create_dialogues", "create_final_script")
workflow.add_edge("create_final_script", "review_script")
workflow.add_conditional_edges("review_script", revision_router, {
    "revise_script": "revise_script"
})
workflow.add_edge("revise_script", "end")
workflow.add_edge("review_script", "end")

# గ్రాఫ్‌ని కంపైల్ చేయడం
app = workflow.compile()

# గ్రాఫ్‌ని రన్ చేయడం
inputs = {
    "genre": "యాక్షన్ థ్రిల్లర్",
    "theme": "న్యాయం కోసం పోరాటం",
    "setting": "ఆధునిక హైదరాబాద్",
    "characters": None,
    "plot_outline": None,
    "act1": None,
    "act2": None,
    "act3": None,
    "dialogues": None,
    "final_script": None,
    "feedback": None,
    "revision_needed": False
}

for output in app.stream(inputs):
    if "final_script" in output and output["final_script"] is not None and not output["revision_needed"]:
        print("\nతెలుగు సినిమా స్క్రిప్ట్ జనరేట్ చేయబడింది!")
        print(f"\nజానర్: {output['genre']}")
        print(f"థీమ్: {output['theme']}")
        print(f"సెట్టింగ్: {output['setting']}")
        
        if "feedback" in output and output["feedback"] is not None:
            feedback = output["feedback"]
            print("\nస్క్రిప్ట్ రివ్యూ:")
            print(f"ఓవరాల్ రేటింగ్: {feedback['overall_rating']}/10")
            print(f"స్టోరీ స్ట్రక్చర్: {feedback['story_structure']}")
            print(f"కథాపాత్రల డెవలప్‌మెంట్: {feedback['character_development']}")
            print(f"డైలాగ్స్: {feedback['dialogues']}")
        
        print("\nస్క్రిప్ట్ మొదటి 500 అక్షరాలు:")
        print(output["final_script"][:500] + "...")

## ముగింపు

ఈ ట్యుటోరియల్‌లో, మనం LangChain లో LangGraph గురించి నేర్చుకున్నాము:

1. LangGraph అంటే ఏమిటో మరియు దాని ప్రాముఖ్యత గురించి తెలుసుకున్నాము
2. LangGraph లో కీ కాన్సెప్ట్స్‌ని చూశాము: నోడ్స్, ఎడ్జెస్, స్టేట్, మరియు ట్రాన్సిషన్స్
3. సింపుల్ LangGraph ఎగ్జాంపుల్‌ని చూశాము
4. మల్టి-స్టెప్ LangGraph ఎగ్జాంపుల్‌ని చూశాము
5. మల్టి-ఏజెంట్ LangGraph ఎగ్జాంపుల్‌ని చూశాము
6. LangGraph ని ఉపయోగించి ఒక రియల్-వరల్డ్ ఉదాహరణను చూశాము: తెలుగు సినిమా స్క్రిప్ట్ జనరేటర్

LangGraph అనేది LangChain ఎకోసిస్టమ్‌లో చాలా శక్తివంతమైన టూల్, ఇది స్టేట్‌ఫుల్ మల్టి-ఏజెంట్ సిస్టమ్స్‌ని బిల్డ్ చేయడానికి ఉపయోగపడుతుంది. ఇది గ్రాఫ్ థియరీ కాన్సెప్ట్స్‌ని ఉపయోగించి, కాంప్లెక్స్ వర్క్‌ఫ్లోలను మాడల్ చేయడానికి మరియు ఎగ్జిక్యూట్ చేయడానికి సహాయపడుతుంది.

తదుపరి ట్యుటోరియల్‌లో, మనం LangChain లో RAG (Retrieval-Augmented Generation) గురించి నేర్చుకుంటాము.